## Start

In [3]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql 
    CREATE TABLE db1.buildings_st_partner
    (
        `parent_uuid` String,
        `installation_point_id` Int64,
        `report_date` Date
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/buildings_st_partner/year=*/month=*/*.csv','CSVWithNames')
    PARTITION BY report_date
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql 
    CREATE TABLE db1.buildings_st_partner_ch
    (
        `parent_uuid` String,
        `installation_point_id` Int64,
        `report_date` Date
    )
    ENGINE = MergeTree()
    ORDER BY installation_point_id
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.buildings_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.buildings_st_partner_ch AS
    SELECT
        *
    FROM db1.buildings_st_partner
    """

ch.query_run(query_text)

___
## Tools
___
### query


In [4]:
query_text = """--sql
    SELECT
        *
    FROM db1.buildings_st_partner_ch
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


,parent_uuid,installation_point_id,report_date
0,988554d7-b9ce-4396-838f-14bde06f67de,0,2025-09-15
1,fb6eeeb6-35d9-47dd-992f-2562323f6d57,0,2025-09-15
2,2c71b664-53bd-497d-8eaa-8fb050370be0,0,2025-09-15
3,5b17fb1f-0f54-404f-a24b-cd37a25c390d,0,2025-09-15
4,14eecf4c-e43d-4bd3-8402-b55773e8161f,0,2025-09-15
5,35e769f4-49ab-4742-8bc8-ce836320f039,0,2025-09-15
6,d371b59f-2c67-448b-9c9c-113db1f9e537,0,2025-09-15
7,9241495d-8330-4ec2-8d5f-7f9bda2fa1b4,0,2025-09-15
8,3ea9d6e8-bc07-427e-a490-9c928cde6728,0,2025-09-15
9,ec3b8739-8e5b-4d81-b1fd-fa6049def62d,0,2025-09-15


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.buildings_st_partner_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [2]:
query_text = """--sql
    DROP TABLE db1.buildings_st_partner_mv
    """

ch.query_run(query_text)

""


### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.buildings_st_partner_ch
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.buildings_st_partner_mv
"""

ch.query_run(query_text)